# 基于概率论的分类方法:朴素贝叶斯 

### 使用Python进行文本分类


计算条件概率的方法:贝叶斯准则
$$
p(c|x) = \frac{p(x|c)p(c)}{p(x)}
$$

In [17]:
'''词表到向量的转换函数
'''
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  #1 代表侮辱性文字
    return postingList,classVec        

In [18]:
def createVocabList(dataSet):
    vocabSet = set()
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

In [19]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec        

In [20]:
listOPosts, listClasses = loadDataSet()
listOPosts
listOPosts[0]

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']

In [21]:
myVocabList = createVocabList(listOPosts)
print(myVocabList)

['is', 'quit', 'dalmation', 'food', 'to', 'worthless', 'love', 'mr', 'him', 'park', 'take', 'I', 'not', 'stupid', 'dog', 'flea', 'help', 'maybe', 'posting', 'how', 'please', 'problems', 'licks', 'stop', 'cute', 'so', 'has', 'garbage', 'ate', 'buying', 'my', 'steak']


In [22]:
print(setOfWords2Vec(myVocabList, listOPosts[1]))

[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### 训练算法:从词向量计算概率
$$
p(c_1|w) = \frac{p(w|c_1)p(c_1)}{p(w)}
$$


In [23]:
import numpy as np
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)  #p(c_1)
    p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)
    p0Denom = 0.0; p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom
    p0Vect = p0Num/p0Denom   #两个类别的概率向量
    return p0Vect, p1Vect, pAbusive

In [24]:
listOPosts, listClasses = loadDataSet()

In [25]:
myVocabList = createVocabList(listOPosts)
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [26]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [27]:
p1V

array([0.        , 0.05263158, 0.        , 0.05263158, 0.05263158,
       0.10526316, 0.        , 0.        , 0.05263158, 0.05263158,
       0.05263158, 0.        , 0.05263158, 0.15789474, 0.10526316,
       0.        , 0.        , 0.05263158, 0.05263158, 0.        ,
       0.        , 0.        , 0.        , 0.05263158, 0.        ,
       0.        , 0.        , 0.05263158, 0.        , 0.05263158,
       0.        , 0.        ])

#### 测试算法: 根据现实情况修改分类器

&nbsp;&nbsp;&nbsp;&nbsp;利用贝叶斯分类器对文档进行分类时,要计算多个概率的乘积以获得文档属于某个类别的概
率,即计算 $p(w_0|1)p(w_1|1)p(w_2|2)$.如果其中一个概率值为0,那么最后的乘积也为0。为降低这种影响,可以将所有词的出现数初始化为1,并将分母初始化为2。
&nbsp;&nbsp;&nbsp;&nbsp;由于大部分因子都非常小,所以程序会下溢出,修改为$ln(f(x))$


In [28]:
#修改后的计算词出现概率
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)  #p(c_1)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)
    p0Denom = 2.0; p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)
    p0Vect = np.log(p0Num/p0Denom)   #两个类别的概率向量
    return p0Vect, p1Vect, pAbusive

In [52]:
'''朴素贝叶斯分类函数
'''
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)  #取对数后变为相加
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, "classified as:", classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, "classified as:", classifyNB(thisDoc, p0V, p1V, pAb))

In [53]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


In [54]:
'''朴素贝叶斯词袋模型
'''
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

## 示例: 使用朴素贝叶斯过滤垃圾邮件

In [72]:
'''文件解析及完整的垃圾邮件测试函数
'''
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList = []; classList = []; fullText = []
    for i in range(1, 26):
        wordList = textParse(open('dataset/email/spam/%d.txt' % i, "rb").read().decode('GBK', 'ignore'))
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('dataset/email/ham/%d.txt' % i, "rb").read().decode('GBK', 'ignore'))
        docList.append(wordList)   #邮件列表
        fullText.extend(wordList)    #词集 1762
        classList.append(0)

    vocabList = createVocabList(docList)    #694
    trainingSet = list(range(50)); testSet = []
    #留存交叉验证
    for i in range(10):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = []; trainClasses = []

    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != \
        classList[docIndex]:
            errorCount += 1
    print("the error rate is: ", float(errorCount)/len(testSet))
spamTest()

the error rate is:  0.1


/home/jimmyli/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


## 示例:使用朴素贝叶斯分类器从个人广告中获取区域倾向

In [73]:
import feedparser

ModuleNotFoundError: No module named 'feedparser'